## 為資策會(iii)提供的樣本 建立ASG物件和node, steplist等檔案

In [1]:
import re
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
from collections import Counter
import pandas as pd
import json
import pickle
from typing import Dict, List, get_type_hints
from IPython.display import display
import time

''' 志剛寫的模組 '''
from ASG import AttackGraph, Node, Edge, FileTable
import Utility

In [2]:
'''
Load 樣本:
路徑為 "C ASG/trace_iii/家族資料夾/樣本資料夾/側錄檔案"
事先過濾掉不需要的家族
取出 Family name
把 malware sample 錄好，產出 ASG pkl, unique steplist, node, regex，放在 Family folder 底下
輸出路徑為 "C ASG statistics 1115ver/saved_pkl/家族資料夾/輸出檔案"
'''

'\nLoad 樣本:\n路徑為 "C ASG/trace_iii/家族資料夾/樣本資料夾/側錄檔案"\n事先過濾掉不需要的家族\n取出 Family name\n把 malware sample 錄好，產出 ASG pkl, unique steplist, node, regex，放在 Family folder 底下\n輸出路徑為 "C ASG statistics 1115ver/saved_pkl/家族資料夾/輸出檔案"\n'

In [3]:
def get_trace_dirs():
    ''' return relative path of traces (sample folders). \n
    Output:\n @sample_paths - relative path of traces. \n
    @family_namedict - 資料夾名稱對應到 family name 的 dict '''
    tarce_root_path = f'../C ASG/trace_iii/'
    ignore_family = []
    family_paths = next(os.walk(tarce_root_path))[1] # 取得subfolder的名稱
    family_namedict = {}
    print('get_trace_dirs()::numOfFamily:', len(family_paths), family_paths)
    sample_paths = []
    for f in family_paths:
        if f in ignore_family:
            continue
        sp = [x[0] for x in os.walk(f"{tarce_root_path}/{f}")][1:]  # 第一個elem是root_path，捨去
        # print(s)
        sample_paths.extend(sp)
        # extract the family name for dir name
        fname = f.split('_')[0]
        family_namedict[f] = fname
        for s in sp:
            family_namedict[s] = fname
     
    return sample_paths, family_namedict

def build(sample_dir: str, output_filename: str='', save_file: bool=True, draw: bool=False):
    ''' build ASG. Read from a trace PATH, store output image at `output/family/sample.svg`. Return `AttackGraph`.'''
    # path = f'../C ASG/trace/{family_dir}/{sample_dir}' # trace log folder for a sample
    path = sample_dir
    if not os.path.isdir(path):
        print(f'  error, path not extst: {path}')
        return None
    # print('  path:', path)
    output_path = path.replace('trace', 'output')
    if len(output_filename):
        output_path = output_path.replace(sample_dir, output_filename)

    # Create Attack Scenario Graph
    graph = AttackGraph(path)
    graph.create()

    return graph

sample_paths, family_namedict = get_trace_dirs()

get_trace_dirs()::numOfFamily: 24 ['642d7f5c4baced8d52cb8b3a', 'AlienReverse_642d7a6c7f12000277e1c88d', 'AvosLocker_61e6e07fe17676b3e61f7052', 'BlackSuit_6462497daf5d5114cdca1a64', 'BPFDoor_627b895a61f9e9b2bc4ecb92', 'BPFDoor_62e8e78bedb06157297a08b6', 'ChinaZ__642311f7791b9d10b49e466e', 'Cl0P_642ab999af92a5fe2c205d8d', 'Conti_645d0a843536d3878d95d18d', 'DarkRadiation_60cb61e79c2d51ea3b16be1c', 'Defray777_603d19ff5e6680fa73bbf7b3', 'Demonbot_60f0059134f2d6364de00c85', 'EnemyBot_625018546137a10d9ebc8ca3', 'EvilGnome_5d2f78275d79002092e0738b', 'FontOnLake_616d81e373e59b8f75f96586', 'HCRootkit_614d8aadc93405bd8c396125', 'HiddenWasp_5cee9b54081b268fd1cf9b39', 'KEYPLUG_642d7cb2e959b168626a5310', 'KEYPLUG_642ec50c398e28491b3a7626', 'Lightning_Framework_62d93453f00b94b4e7b7274e', 'Mirai_628ba92b2d40b12383c1344d', 'Moose_5564c9edb45ff543aee5b437', 'Royal_63f640bd42bcd8428c6f3e12', 'XMRigMiner_63ac153614c9db1f6699fa19']


Cl0P_642ab99 跑太久..跳過

In [4]:
for i, sp in enumerate(sample_paths):
    print(i, sp)
    if i <= 33:
        continue
    try:
        asg = None
        asg = build(sp)
    except:
        time.sleep(0.1)
        asg = build(sp)  # 常常有 io 錯誤所以給他一次 rerun 機會

    # 取得 objects 和 steplist
    if asg == None:
        continue
    seen_node_S, seen_node_O = Utility.create_set_of_objects(asg)
    steplist = Utility.get_sorted_uni_step(asg)

    # 儲存 asg 物件, destobjects, steplist
    smaple_name = sp.split('\\')[-1]
    family_name = family_namedict[sp]
    family_path = f'./saved_pkl/{family_name}'
    Path(family_path).mkdir(parents=True, exist_ok=True)
    with open(f'{family_path}/{smaple_name}.pkl', 'wb') as outp:
        pickle.dump(asg, outp, pickle.HIGHEST_PROTOCOL)
    with open(f'{family_path}/{smaple_name}_object.txt', 'w') as outf:
        outf.writelines(line + '\n' for line in seen_node_O)
    with open(f'{family_path}/{smaple_name}_steplist.txt', 'w') as outf:
        outf.writelines(f"{step[0]:8} -> {step[2]:15} -> {step[1]}" + '\n' for step in steplist)
    # break

0 ../C ASG/trace_iii//642d7f5c4baced8d52cb8b3a\b711579e33b0df2143c7cb61246233c7f9b4d53db6a048427a58c0295d8daf1c
1 ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\187b6a4c6bc379c183657d8eafc225da53ab8f78ac192704b713cc202cf89a17
Strace merge path: ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\187b6a4c6bc379c183657d8eafc225da53ab8f78ac192704b713cc202cf89a17
2 ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\1f9e4bfb25622eab6c33da7da9be6c51cf8bf1a284ee1c1703a3cee445bc8cd9
Strace merge path: ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\1f9e4bfb25622eab6c33da7da9be6c51cf8bf1a284ee1c1703a3cee445bc8cd9
3 ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\22fd67457274635db7dd679782e002009363010db66523973b4748d5778b1a2a
Strace merge path: ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\22fd67457274635db7dd679782e002009363010db66523973b4748d5778b1a2a
4 ../C ASG/trace_iii//AlienReverse_642d7a6c7f12000277e1c88d\2801a3cc5aed8ecb391a9638a3c